# LLM "No Hint" Migration 
This notebook demonstrates how an LLM (using Kai's ModelProvider) performs when presented with a Spring Security migration issue without a hint.

Installing pre requisites and configuring Kai with `granite-8b-code-instruct-128k`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [ ]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 

# Initialize the model provider 
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "granite-8b-code-instruct-128k",
        "base_url": "https://granite-8b-code-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1",
        "api_key": os.getenv("OPENAI_API_KEY"),},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [9]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [10]:
message=""""\

  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
"""

In [11]:
prompt = f"""
The following java code needs to be migrated from springboot2 to springboot3. 

```java
{before_code}
```
The issue has been identified by an analysis rule

{message}

Can you provide a refactored version of this code that is compatible with springboot3?
Also, create an hint based on your suggestions.

"""

In [12]:

response = await rendered_llm_call(prompt)

Refactored code:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(HttpSecurity http) throws Exception {
        http.authorizeRequests()
            .anyRequest().authenticated()
            .and()
            .httpBasic();
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
```

Hint:

The `configure(AuthenticationManagerBuilder)` method is no longer necessary in Spring Security 5.7. Instead, you should define a `UserDetailsService` and `PasswordEncoder` bean. The `configure(HttpSecurity)` method can be used to configure authentication and authorization settings.


### Evaluation

| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  Yes           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           | Partially Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  No           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | No           |

